In [ ]:
# CELL 0: SETUP
import pandas as pd
import numpy as np
import time
import os
import joblib
import warnings
from sklearn.exceptions import UndefinedMetricWarning

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer 
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.compose import ColumnTransformer 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix, make_scorer

warnings.filterwarnings("ignore", category=UndefinedMetricWarning, module="sklearn.metrics._classification")
warnings.filterwarnings("ignore", category=UserWarning, module="optuna.samplers")
warnings.filterwarnings("ignore", category=FutureWarning)

def print_metrics(metrics_dict, prefix=""):
    print(f"{prefix}AUC: {metrics_dict.get('roc_auc', float('nan')):.4f}, F1: {metrics_dict.get('f1', float('nan')):.4f}, "
          f"Precision: {metrics_dict.get('precision', float('nan')):.4f}, Recall: {metrics_dict.get('recall', float('nan')):.4f}")
    if 'confusion_matrix' in metrics_dict:
        cm_flat = metrics_dict['confusion_matrix'].ravel()
        if len(cm_flat) == 4: print(f"{prefix}CM (tn,fp,fn,tp): {cm_flat}")

RANDOM_STATE = 42
PRIMARY_METRIC_NAME = 'f1'
N_CV_SPLITS = 3

SCORING_FOR_SKLEARN_CV_METHODS = {
    'roc_auc': 'roc_auc',
    'f1': 'f1',
    'precision': 'precision',
    'recall': 'recall'
}
REFIT_METRIC_FOR_SKLEARN_CV = PRIMARY_METRIC_NAME
if REFIT_METRIC_FOR_SKLEARN_CV not in SCORING_FOR_SKLEARN_CV_METHODS:
    raise ValueError(f"REFIT_METRIC_FOR_SKLEARN_CV ('{REFIT_METRIC_FOR_SKLEARN_CV}') "
                     f"must be a key in SCORING_FOR_SKLEARN_CV_METHODS.")

print(f"Global Settings: Primary Metric (Refit): {PRIMARY_METRIC_NAME.upper()}, CV Folds: {N_CV_SPLITS}")
print(f"Scoring for Grid/RandomizedSearch: {SCORING_FOR_SKLEARN_CV_METHODS}, Refit on: {REFIT_METRIC_FOR_SKLEARN_CV}")

BASE_DIR = "."
SAVED_TUNED_MODELS_DIR = os.path.join(BASE_DIR, "saved_tuned_models")
SAVED_OVERALL_BEST_MODELS_DIR = os.path.join(BASE_DIR, "saved_overall_best_models")
RESULTS_FILE_PATH = os.path.join(BASE_DIR, "all_models_results_persistent.joblib")
SPLITS_FILE_PATH = os.path.join(BASE_DIR, "data_splits_storage_persistent.joblib") # Single file backup

# --- NEW: Directory for individual data splits per target ---
CLASSIFICATION_MODELS_DATA_DIR = os.path.join(BASE_DIR, "classification_models_data")

# Assuming DATA_SPLITS_DIR_FOR_SHAP is defined if used, or remove if not.
# For example, it might be: DATA_SPLITS_DIR_FOR_SHAP = os.path.join(BASE_DIR, "shap_data_splits")
# os.makedirs(DATA_SPLITS_DIR_FOR_SHAP, exist_ok=True) # Ensure this still exists if SHAP uses it (original line)
os.makedirs(SAVED_TUNED_MODELS_DIR, exist_ok=True)
os.makedirs(SAVED_OVERALL_BEST_MODELS_DIR, exist_ok=True)
os.makedirs(CLASSIFICATION_MODELS_DATA_DIR, exist_ok=True) # Create the new directory

df = pd.read_csv("../data/cleaned_choreographies.csv")


feature_columns = ["timeDuration", "nMovements", "movementsDifficulty", "robotSpeech",
                   "acrobaticMovements", "movementsRepetition", "movementsTransitionsDuration",
                   "humanMovements", "balance", "speed", "bodyPartsCombination", "musicBPM",
                   "sameStartEndPositionPlace", "headMovement", "armsMovement", "handsMovement",
                   "legsMovement", "feetMovement", "musicGenre_electronic", "musicGenre_folk",
                   "musicGenre_indie", "musicGenre_latin", "musicGenre_pop", "musicGenre_rap",
                   "musicGenre_rock"]
target_columns = [c for c in df.columns if c not in feature_columns and c not in ["ChoreographyID", "HumanScore"]] if not df.empty else []

if not df.empty:
    df_binary = df.copy()
    df_binary[target_columns] = (df_binary[target_columns] >= 4).astype(int)
    X_full = df_binary[feature_columns].copy()
    print(f"Data loaded: {df_binary.shape}, Features: {X_full.shape[1]}, Targets: {len(target_columns)}")


def load_joblib_file(path, description):
    try:
        data = joblib.load(path)
        print(f"Loaded existing `{description}`.")
        return data
    except FileNotFoundError:
        print(f"No existing `{description}` file. Initializing empty.")
        return {}
    except Exception as e:
        print(f"Error loading `{description}`: {e}. Initializing empty.")
        return {}

all_models_results = load_joblib_file(RESULTS_FILE_PATH, "all_models_results")
data_splits_storage = load_joblib_file(SPLITS_FILE_PATH, "data_splits_storage") # Still use this for status tracking

print("\nSETUP CELL 0 COMPLETED.\n" + "="*50 + "\n")

Global Settings: Primary Metric (Refit): F1, CV Folds: 3
Scoring for Grid/RandomizedSearch: {'roc_auc': 'roc_auc', 'f1': 'f1', 'precision': 'precision', 'recall': 'recall'}, Refit on: f1
Data loaded: (8563, 32), Features: 25, Targets: 7
No existing `all_models_results` file. Initializing empty.
No existing `data_splits_storage` file. Initializing empty.

SETUP CELL 0 COMPLETED.



In [5]:
# CELL 0A: UNIFIED DATA SPLITTING & SAVING (with CSV output)
# Performs one-time 80% train / 20% holdout split per target if not already done.
# Also saves these splits to individual CSV files per target.

print("\n--- Unified Data Splitting (80% Train / 20% Holdout) & CSV Saving ---")

if 'target_columns' not in globals() or not target_columns:
    print("ERROR: 'target_columns' is not defined or empty. Skipping data splitting.")
# Ensure CLASSIFICATION_MODELS_DATA_DIR is defined (usually in CELL 0)
elif 'CLASSIFICATION_MODELS_DATA_DIR' not in globals():
    print("ERROR: 'CLASSIFICATION_MODELS_DATA_DIR' is not defined. Please run CELL 0 first.")
else:
    targets_requiring_processing = [] # Targets that need splitting or saving to CSV
    for tc in target_columns:
        safe_tc_name = "".join(c if c.isalnum() else "_" for c in tc)
        target_specific_save_dir = os.path.join(CLASSIFICATION_MODELS_DATA_DIR, safe_tc_name)
        
        # Check if CSV files exist on disk
        csv_files_exist_on_disk = (
            os.path.exists(os.path.join(target_specific_save_dir, "X_train.csv")) and
            os.path.exists(os.path.join(target_specific_save_dir, "y_train.csv")) and
            os.path.exists(os.path.join(target_specific_save_dir, "X_holdout.csv")) and
            os.path.exists(os.path.join(target_specific_save_dir, "y_holdout.csv"))
        )
        
        # Check if split exists in data_splits_storage
        split_in_storage_valid = (
            tc in data_splits_storage and
            isinstance(data_splits_storage[tc], dict) and
            data_splits_storage[tc].get('status') == 'Split successful' and
            all(k in data_splits_storage[tc] for k in ['X_train', 'X_holdout', 'y_train', 'y_holdout'])
        )

        # If CSVs don't exist, or if storage path doesn't match, or if status indicates previous save fail
        if not csv_files_exist_on_disk or \
           (split_in_storage_valid and data_splits_storage[tc].get('saved_to_dir_path') != target_specific_save_dir) or \
           (split_in_storage_valid and data_splits_storage[tc].get('saved_to_dir_path') == 'FailedToSaveCSV'):
            targets_requiring_processing.append(tc)
        elif not split_in_storage_valid: # If storage is invalid/missing but CSVs might exist (less likely path)
             targets_requiring_processing.append(tc)


    if not targets_requiring_processing and data_splits_storage and any(data_splits_storage):
        # This condition might need refinement if only some targets were processed before
        # For simplicity, if the list is empty, assume all are done if storage is not empty
        all_confirmed = True
        for tc_check in target_columns:
            safe_tc_name_check = "".join(c if c.isalnum() else "_" for c in tc_check)
            target_specific_save_dir_check = os.path.join(CLASSIFICATION_MODELS_DATA_DIR, safe_tc_name_check)
            if not (data_splits_storage.get(tc_check, {}).get('status') == 'Split successful' and \
                    data_splits_storage.get(tc_check, {}).get('saved_to_dir_path') == target_specific_save_dir_check):
                all_confirmed = False
                break
        if all_confirmed:
            print("All required data splits are already present, valid in storage, and saved to individual CSV directories. Skipping re-processing.")
            targets_to_process_now = []
        else:
            # If not all confirmed, re-evaluate based on targets_requiring_processing
            if not targets_requiring_processing: # Should not happen if all_confirmed is False, but as a safeguard
                 print("Re-evaluating all targets for splitting/saving as consistency check.")
                 targets_to_process_now = target_columns
            else:
                 print(f"CSV Splits/saves needed for: {targets_requiring_processing}")
                 targets_to_process_now = targets_requiring_processing

    elif not data_splits_storage or not any(data_splits_storage): # If storage empty
        print("`data_splits_storage` is empty. Processing all targets for splitting/saving.")
        targets_to_process_now = target_columns
    else: # Some targets need processing
        print(f"CSV Splits/saves needed for: {targets_requiring_processing}")
        targets_to_process_now = targets_requiring_processing


    if targets_to_process_now: # Only proceed if there are targets to process
        for target_name in targets_to_process_now:
            print(f"\nProcessing target for splitting/saving CSVs: {target_name}")
            
            safe_target_name_for_dir = "".join(c if c.isalnum() else "_" for c in target_name)
            current_target_save_dir = os.path.join(CLASSIFICATION_MODELS_DATA_DIR, safe_target_name_for_dir)

            X_train_s, y_train_s, X_holdout_s, y_holdout_s = None, None, None, None
            perform_actual_split = True

            # Check if we can use existing splits from data_splits_storage
            if target_name in data_splits_storage and data_splits_storage[target_name].get('status') == 'Split successful':
                print(f"  Valid split for '{target_name}' found in `data_splits_storage`. Checking CSV save status.")
                
                csv_files_on_disk = (
                    os.path.exists(os.path.join(current_target_save_dir, "X_train.csv")) and
                    os.path.exists(os.path.join(current_target_save_dir, "y_train.csv")) and
                    os.path.exists(os.path.join(current_target_save_dir, "X_holdout.csv")) and
                    os.path.exists(os.path.join(current_target_save_dir, "y_holdout.csv"))
                )
                if csv_files_on_disk and data_splits_storage[target_name].get('saved_to_dir_path') == current_target_save_dir:
                    print(f"  CSV files for '{target_name}' already saved to '{current_target_save_dir}'. Skipping.")
                    perform_actual_split = False # Already split and CSVs saved
                else:
                    print(f"  CSV files for '{target_name}' not found or path mismatch in '{current_target_save_dir}'. Will save from storage if available.")
                    X_train_s = data_splits_storage[target_name]['X_train']
                    y_train_s = data_splits_storage[target_name]['y_train']
                    X_holdout_s = data_splits_storage[target_name]['X_holdout']
                    y_holdout_s = data_splits_storage[target_name]['y_holdout']
                    perform_actual_split = False # Data is ready from storage, just needs saving to CSV
            
            if perform_actual_split:
                if df_binary.empty or target_name not in df_binary.columns:
                    print(f"  Skipping '{target_name}' due to empty/missing data in df_binary.")
                    data_splits_storage[target_name] = {'status': 'Skipped - data missing', 'saved_to_dir_path': None}
                    continue
                y_full_target = df_binary[target_name].copy()

                if y_full_target.nunique() < 2:
                    print(f"  WARNING: Target '{target_name}' has only one class. Skipping.")
                    data_splits_storage[target_name] = {'status': 'Skipped - single class', 'saved_to_dir_path': None}
                    continue

                min_class_samples = y_full_target.value_counts().min()
                if min_class_samples < 2 : 
                    print(f"  WARNING: Smallest class in '{target_name}' ({min_class_samples}) too small for initial stratification. Skipping.")
                    data_splits_storage[target_name] = {'status': f'Skipped - too few samples for stratification ({min_class_samples})', 'saved_to_dir_path': None}
                    continue
                
                try:
                    X_train_s, X_holdout_s, y_train_s, y_holdout_s = train_test_split(
                        X_full, y_full_target, test_size=0.2, random_state=RANDOM_STATE, stratify=y_full_target
                    )
                    # Store in-memory pandas objects in data_splits_storage
                    data_splits_storage[target_name] = {
                        'X_train': X_train_s, 'X_holdout': X_holdout_s,
                        'y_train': y_train_s, 'y_holdout': y_holdout_s,
                        'status': 'Split successful',
                        'saved_to_dir_path': None 
                    }
                    print(f"  Split successful for '{target_name}'. Train: {X_train_s.shape[0]}, Holdout: {X_holdout_s.shape[0]}")
                except ValueError as e:
                    print(f"  ERROR during train_test_split for {target_name}: {e}")
                    data_splits_storage[target_name] = {'status': f'Skipped - split error: {e}', 'saved_to_dir_path': None}
                    continue
            
            # Save to individual CSV files if split was successful (either just now or from storage and not yet saved as CSV)
            if data_splits_storage.get(target_name, {}).get('status') == 'Split successful':
                # Ensure X_train_s etc. are populated if perform_actual_split was False but CSVs needed saving
                if X_train_s is None: # This happens if split was in storage, but files were not on disk, and we didn't re-assign above
                    X_train_s = data_splits_storage[target_name]['X_train']
                    y_train_s = data_splits_storage[target_name]['y_train']
                    X_holdout_s = data_splits_storage[target_name]['X_holdout']
                    y_holdout_s = data_splits_storage[target_name]['y_holdout']

                try:
                    os.makedirs(current_target_save_dir, exist_ok=True)
                    # Save as CSV
                    X_train_s.to_csv(os.path.join(current_target_save_dir, "X_train.csv"), index=False)
                    y_train_s.to_csv(os.path.join(current_target_save_dir, "y_train.csv"), index=False, header=True) # y_train_s is a Series, .name will be used as header
                    X_holdout_s.to_csv(os.path.join(current_target_save_dir, "X_holdout.csv"), index=False)
                    y_holdout_s.to_csv(os.path.join(current_target_save_dir, "y_holdout.csv"), index=False, header=True) # y_holdout_s is a Series

                    print(f"  Data splits for '{target_name}' saved as CSV to directory: {current_target_save_dir}")
                    data_splits_storage[target_name]['saved_to_dir_path'] = current_target_save_dir 
                    data_splits_storage[target_name]['save_format'] = 'csv' # Indicate format
                except Exception as e:
                    print(f"  ERROR saving CSV splits to directory for {target_name}: {e}")
                    data_splits_storage[target_name]['saved_to_dir_path'] = 'FailedToSaveCSV'
            elif data_splits_storage.get(target_name, {}).get('status') != 'Split successful':
                 print(f"  Skipping CSV save for '{target_name}' as split was not successful (status: {data_splits_storage.get(target_name, {}).get('status')}).")


        try:
            joblib.dump(data_splits_storage, SPLITS_FILE_PATH)
            print(f"\n`data_splits_storage` (with statuses and dir paths) updated and saved to {SPLITS_FILE_PATH}")
        except Exception as e:
            print(f"\nError saving `data_splits_storage`: {e}")
    else: # if not targets_to_process_now
        print("No targets required splitting or CSV saving in this run.")


print("\n--- Unified Data Splitting and CSV Directory Saving Completed ---\n" + "="*50 + "\n")


--- Unified Data Splitting (80% Train / 20% Holdout) & CSV Saving ---
`data_splits_storage` is empty. Processing all targets for splitting/saving.

Processing target for splitting/saving CSVs: EvaluationChoreographyStoryTelling
  Split successful for 'EvaluationChoreographyStoryTelling'. Train: 6850, Holdout: 1713
  Data splits for 'EvaluationChoreographyStoryTelling' saved as CSV to directory: ./classification_models_data/EvaluationChoreographyStoryTelling

Processing target for splitting/saving CSVs: EvaluationChoreographyRhythm
  Split successful for 'EvaluationChoreographyRhythm'. Train: 6850, Holdout: 1713
  Data splits for 'EvaluationChoreographyRhythm' saved as CSV to directory: ./classification_models_data/EvaluationChoreographyRhythm

Processing target for splitting/saving CSVs: EvaluationChoreographyMovementTechnique
  Split successful for 'EvaluationChoreographyMovementTechnique'. Train: 6850, Holdout: 1713
  Data splits for 'EvaluationChoreographyMovementTechnique' saved a

In [ ]:
# CELL 1: LOGISTIC REGRESSION (GridSearchCV)
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold

model_name_lr = "LogisticRegression_GridCV"
if model_name_lr not in all_models_results:
    all_models_results[model_name_lr] = {}

print(f"\n--- Training Model: {model_name_lr} ---")

if 'target_columns' not in globals() or not target_columns:
    print(f"ERROR: 'target_columns' not defined/empty. Skipping {model_name_lr} training.")
else:
    for target_name in target_columns:
        print(f"\n===== Training {model_name_lr} for Target: {target_name} =====")

        if target_name not in data_splits_storage or data_splits_storage[target_name]['status'] != 'Split successful':
            status = data_splits_storage.get(target_name, {}).get('status', 'Split data not found or failed')
            print(f"  Skipping '{target_name}'. Reason: {status}")
            all_models_results[model_name_lr][target_name] = {'status': status}
            continue

        X_train_80pct = data_splits_storage[target_name]['X_train']
        y_train_80pct = data_splits_storage[target_name]['y_train']
        X_holdout_20pct = data_splits_storage[target_name]['X_holdout']
        y_holdout_20pct = data_splits_storage[target_name]['y_holdout']

        min_class_count_train = y_train_80pct.value_counts().min()
        if min_class_count_train < N_CV_SPLITS:
            print(f"  WARNING: Smallest class ({min_class_count_train}) < N_CV_SPLITS ({N_CV_SPLITS}) for '{target_name}'. Skipping.")
            all_models_results[model_name_lr][target_name] = {
                'status': f'Skipped - too few samples for {N_CV_SPLITS}-Fold CV ({min_class_count_train})'
            }
            continue


        if 'feature_columns' not in globals():
             raise NameError("feature_columns not defined. Please ensure CELL 0 has been run.")

        ohe_cols = [col for col in feature_columns if 'musicGenre_' in col]
        numerical_cols_for_robust_scaling = [col for col in feature_columns if col not in ohe_cols]
        
        # Define the preprocessor using ColumnTransformer
        # n_neighbors for KNNImputer is set to a common default (5). Tune if necessary.
        preprocessor_lr = ColumnTransformer(
            transformers=[
                ('num_processing', Pipeline([
                    ('imputer_num', KNNImputer(n_neighbors=5)),
                    ('scaler', RobustScaler())
                ]), numerical_cols_for_robust_scaling),
                ('ohe_processing', Pipeline([
                    ('imputer_ohe', KNNImputer(n_neighbors=5)) # Impute OHE but do not scale
                ]), ohe_cols)
            ],
            remainder='passthrough' # Ensures any columns not explicitly handled are passed through.
                                    # If feature_columns covers all columns in X_train_80pct, this has no practical effect on them.
        )

        lr_pipeline = Pipeline([
            ('preprocessing', preprocessor_lr),
            ('clf', LogisticRegression(solver='liblinear', class_weight='balanced', random_state=RANDOM_STATE, max_iter=1000))
        ])
        
        # Parameters for tuning the classifier.
        # If you want to tune KNNImputer's n_neighbors, you would add something like:
        # 'preprocessing__num_processing__imputer_num__n_neighbors': [3, 5, 7, 9],
        # 'preprocessing__ohe_processing__imputer_ohe__n_neighbors': [3, 5, 7, 9],
        lr_param_grid = {
            'clf__C': np.logspace(-3, 3, 7),
            'clf__penalty': ['l1', 'l2']
        }

        cv_splitter = StratifiedKFold(n_splits=N_CV_SPLITS, shuffle=True, random_state=RANDOM_STATE)
        search_cv_lr = GridSearchCV(
            lr_pipeline, lr_param_grid, cv=cv_splitter,
            scoring=SCORING_FOR_SKLEARN_CV_METHODS,
            refit=REFIT_METRIC_FOR_SKLEARN_CV,
            verbose=0, n_jobs=-1
        )

        print(f"  Starting GridSearchCV for '{target_name}'...")
        start_time_cv_lr = time.time()
        try:
            search_cv_lr.fit(X_train_80pct, y_train_80pct)
        except Exception as e:
            print(f"  ERROR during GridSearchCV for '{target_name}': {e}")
            all_models_results[model_name_lr][target_name] = {'status': f'Skipped - SearchCV error: {e}'}
            continue
        cv_training_time_lr = time.time() - start_time_cv_lr
        print(f"  GridSearchCV completed in {cv_training_time_lr:.2f}s. Best CV {REFIT_METRIC_FOR_SKLEARN_CV.upper()}: {search_cv_lr.best_score_:.4f}")

        tuned_model_lr = search_cv_lr.best_estimator_
        tuned_model_filename = f"{model_name_lr}_{target_name.replace(' ', '_')}.joblib"
        tuned_model_path = os.path.join(SAVED_TUNED_MODELS_DIR, tuned_model_filename)
        joblib.dump(tuned_model_lr, tuned_model_path)

        y_train_pred_lr = tuned_model_lr.predict(X_train_80pct)
        y_train_proba_lr = tuned_model_lr.predict_proba(X_train_80pct)[:, 1]
        train_set_metrics_lr = {
            'roc_auc': roc_auc_score(y_train_80pct, y_train_proba_lr),
            'f1': f1_score(y_train_80pct, y_train_pred_lr, zero_division=0),
            'precision': precision_score(y_train_80pct, y_train_pred_lr, zero_division=0),
            'recall': recall_score(y_train_80pct, y_train_pred_lr, zero_division=0),
            'confusion_matrix': confusion_matrix(y_train_80pct, y_train_pred_lr)
        }

        y_holdout_pred_lr = tuned_model_lr.predict(X_holdout_20pct)
        y_holdout_proba_lr = tuned_model_lr.predict_proba(X_holdout_20pct)[:, 1]
        holdout_set_metrics_lr = {
            'roc_auc': roc_auc_score(y_holdout_20pct, y_holdout_proba_lr),
            'f1': f1_score(y_holdout_20pct, y_holdout_pred_lr, zero_division=0),
            'precision': precision_score(y_holdout_20pct, y_holdout_pred_lr, zero_division=0),
            'recall': recall_score(y_holdout_20pct, y_holdout_pred_lr, zero_division=0),
            'confusion_matrix': confusion_matrix(y_holdout_20pct, y_holdout_pred_lr)
        }
        print("  Metrics on Hold-Out Set (20%):"); print_metrics(holdout_set_metrics_lr, prefix="  Holdout: ")

        results_df_lr = pd.DataFrame(search_cv_lr.cv_results_)
        best_index_lr = search_cv_lr.best_index_
        fold_metrics_summary_lr = {}
        for metric_key_cv_results in SCORING_FOR_SKLEARN_CV_METHODS.keys():
            fold_metrics_summary_lr[metric_key_cv_results] = {
                'mean': results_df_lr.iloc[best_index_lr][f'mean_test_{metric_key_cv_results}'],
                'std': results_df_lr.iloc[best_index_lr][f'std_test_{metric_key_cv_results}']
            }

        all_models_results[model_name_lr][target_name] = {
            'status': 'Completed', 'best_hyperparameters': search_cv_lr.best_params_,
            'cv_mean_metrics': fold_metrics_summary_lr,
            f'best_cv_{REFIT_METRIC_FOR_SKLEARN_CV}_score': search_cv_lr.best_score_,
            'train_set_metrics': train_set_metrics_lr, 'holdout_set_metrics': holdout_set_metrics_lr,
            'cv_tuning_time_seconds': cv_training_time_lr,
            'saved_tuned_model_path': tuned_model_path,
        }

    print(f"\n--- Completed All Targets for {model_name_lr} ---")
    try:
        joblib.dump(all_models_results, RESULTS_FILE_PATH)
        print(f"`all_models_results` updated after {model_name_lr}.")
    except Exception as e:
        print(f"Error saving `all_models_results` after {model_name_lr}: {e}")

print("\n" + "="*50 + "\n")


--- Training Model: LogisticRegression_GridCV ---

===== Training LogisticRegression_GridCV for Target: EvaluationChoreographyStoryTelling =====
  Starting GridSearchCV for 'EvaluationChoreographyStoryTelling'...


/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_div

  GridSearchCV completed in 1.78s. Best CV F1: 0.5577
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.6802, F1: 0.5316, Precision: 0.4985, Recall: 0.5695
  Holdout: CM (tn,fp,fn,tp): [796 334 251 332]

===== Training LogisticRegression_GridCV for Target: EvaluationChoreographyRhythm =====
  Starting GridSearchCV for 'EvaluationChoreographyRhythm'...


/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_div

  GridSearchCV completed in 0.29s. Best CV F1: 0.6043
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.6776, F1: 0.5973, Precision: 0.6062, Recall: 0.5887
  Holdout: CM (tn,fp,fn,tp): [708 278 299 428]

===== Training LogisticRegression_GridCV for Target: EvaluationChoreographyMovementTechnique =====
  Starting GridSearchCV for 'EvaluationChoreographyMovementTechnique'...


...

  GridSearchCV completed in 0.28s. Best CV F1: 0.5022
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.6510, F1: 0.4927, Precision: 0.4074, Recall: 0.6234
  Holdout: CM (tn,fp,fn,tp): [832 419 174 288]

===== Training LogisticRegression_GridCV for Target: EvaluationChoreographyHumanCharacterization =====
  Starting GridSearchCV for 'EvaluationChoreographyHumanCharacterization'...


/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_div

  GridSearchCV completed in 0.27s. Best CV F1: 0.4844
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.5334, F1: 0.4773, Precision: 0.4223, Recall: 0.5487
  Holdout: CM (tn,fp,fn,tp): [491 524 315 383]

===== Training LogisticRegression_GridCV for Target: EvaluationChoreographyHumanReproducibility =====
  Starting GridSearchCV for 'EvaluationChoreographyHumanReproducibility'...


/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_div

  GridSearchCV completed in 0.30s. Best CV F1: 0.6324
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.5321, F1: 0.6368, Precision: 0.7848, Recall: 0.5358
  Holdout: CM (tn,fp,fn,tp): [206 193 610 704]

--- Completed All Targets for LogisticRegression_GridCV ---
`all_models_results` updated after LogisticRegression_GridCV.




In [ ]:
# CELL 2: RANDOM FOREST (GridSearchCV)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


model_name_rf = "RandomForest_GridCV" 
if model_name_rf not in all_models_results:
    all_models_results[model_name_rf] = {}

print(f"\n--- Training Model: {model_name_rf} ---")


rf_param_grid = { 
    'clf__n_estimators': [100, 200, 300], 
    'clf__max_depth': [None, 10, 20],     
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4],
    'clf__max_features': ['sqrt', 'log2'] 
}


if 'target_columns' not in globals() or not target_columns:
    print(f"ERROR: 'target_columns' not defined/empty. Skipping {model_name_rf} training.")
else:
    for target_name in target_columns:
        print(f"\n===== Training {model_name_rf} for Target: {target_name} =====")

        if target_name not in data_splits_storage or data_splits_storage[target_name]['status'] != 'Split successful':
            status = data_splits_storage.get(target_name, {}).get('status', 'Split data not found or failed')
            print(f"  Skipping '{target_name}'. Reason: {status}")
            all_models_results[model_name_rf][target_name] = {'status': status}
            continue

        X_train_80pct = data_splits_storage[target_name]['X_train']
        y_train_80pct = data_splits_storage[target_name]['y_train']
        X_holdout_20pct = data_splits_storage[target_name]['X_holdout']
        y_holdout_20pct = data_splits_storage[target_name]['y_holdout']

        min_class_count_train = y_train_80pct.value_counts().min()
        if min_class_count_train < N_CV_SPLITS:
            print(f"  WARNING: Smallest class ({min_class_count_train}) < N_CV_SPLITS ({N_CV_SPLITS}) for '{target_name}'. Skipping.")
            all_models_results[model_name_rf][target_name] = {
                'status': f'Skipped - too few samples for {N_CV_SPLITS}-Fold CV ({min_class_count_train})'
            }
            continue

        rf_pipeline = Pipeline([
            ('imputer', KNNImputer(n_neighbors=5)),
            ('clf', RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced_subsample', oob_score=False))
        ])

        cv_splitter = StratifiedKFold(n_splits=N_CV_SPLITS, shuffle=True, random_state=RANDOM_STATE)

        search_cv_rf = GridSearchCV( # Changed
            rf_pipeline, param_grid=rf_param_grid, # Changed
            cv=cv_splitter,
            scoring=SCORING_FOR_SKLEARN_CV_METHODS, 
            refit=REFIT_METRIC_FOR_SKLEARN_CV,   
            verbose=0, n_jobs=-1
        )


        
        print(f"  Starting GridSearchCV for '{target_name}'...") 
        start_time_cv_rf = time.time()
        try:
            search_cv_rf.fit(X_train_80pct, y_train_80pct)

        except ValueError as ve: # Catch common issues like parameter errors specifically
             print(f"  ERROR during GridSearchCV for '{target_name}' (ValueError): {ve}")
             all_models_results[model_name_rf][target_name] = {'status': f'Skipped - SearchCV ValueError: {ve}'}
             continue
        except Exception as e:
            print(f"  ERROR during GridSearchCV for '{target_name}': {e}")
            all_models_results[model_name_rf][target_name] = {'status': f'Skipped - SearchCV error: {e}'}
            continue
        cv_tuning_time_rf = time.time() - start_time_cv_rf
       
        print(f"  GridSearchCV completed in {cv_tuning_time_rf:.2f}s. Best CV {REFIT_METRIC_FOR_SKLEARN_CV.upper()}: {search_cv_rf.best_score_:.4f}") # Changed

        tuned_model_rf = search_cv_rf.best_estimator_
        tuned_model_filename = f"{model_name_rf}_{target_name.replace(' ', '_')}.joblib"
        tuned_model_path = os.path.join(SAVED_TUNED_MODELS_DIR, tuned_model_filename)
        joblib.dump(tuned_model_rf, tuned_model_path)

        y_train_pred_rf = tuned_model_rf.predict(X_train_80pct)
        y_train_proba_rf = tuned_model_rf.predict_proba(X_train_80pct)[:, 1]
        train_set_metrics_rf = {
            'roc_auc': roc_auc_score(y_train_80pct, y_train_proba_rf),
            'f1': f1_score(y_train_80pct, y_train_pred_rf, zero_division=0),
            'precision': precision_score(y_train_80pct, y_train_pred_rf, zero_division=0),
            'recall': recall_score(y_train_80pct, y_train_pred_rf, zero_division=0),
            'confusion_matrix': confusion_matrix(y_train_80pct, y_train_pred_rf)
        }

        y_holdout_pred_rf = tuned_model_rf.predict(X_holdout_20pct)
        y_holdout_proba_rf = tuned_model_rf.predict_proba(X_holdout_20pct)[:, 1]
        holdout_set_metrics_rf = {
            'roc_auc': roc_auc_score(y_holdout_20pct, y_holdout_proba_rf),
            'f1': f1_score(y_holdout_20pct, y_holdout_pred_rf, zero_division=0),
            'precision': precision_score(y_holdout_20pct, y_holdout_pred_rf, zero_division=0),
            'recall': recall_score(y_holdout_20pct, y_holdout_pred_rf, zero_division=0),
            'confusion_matrix': confusion_matrix(y_holdout_20pct, y_holdout_pred_rf)
        }
        print("  Metrics on Hold-Out Set (20%):"); print_metrics(holdout_set_metrics_rf, prefix="  Holdout: ")

        results_df_rf = pd.DataFrame(search_cv_rf.cv_results_)
        best_index_rf = search_cv_rf.best_index_
        fold_metrics_summary_rf = {}
        for metric_key_cv_results in SCORING_FOR_SKLEARN_CV_METHODS.keys():
            fold_metrics_summary_rf[metric_key_cv_results] = {
                'mean': results_df_rf.iloc[best_index_rf][f'mean_test_{metric_key_cv_results}'],
                'std': results_df_rf.iloc[best_index_rf][f'std_test_{metric_key_cv_results}']
            }

        all_models_results[model_name_rf][target_name] = {
            'status': 'Completed', 'best_hyperparameters': search_cv_rf.best_params_,
            'cv_mean_metrics': fold_metrics_summary_rf,
            f'best_cv_{REFIT_METRIC_FOR_SKLEARN_CV}_score': search_cv_rf.best_score_,
            'train_set_metrics': train_set_metrics_rf, 'holdout_set_metrics': holdout_set_metrics_rf,
            'cv_tuning_time_seconds': cv_tuning_time_rf,
            'saved_tuned_model_path': tuned_model_path,
        }

    print(f"\n--- Completed All Targets for {model_name_rf} ---")
    try:
        joblib.dump(all_models_results, RESULTS_FILE_PATH)
        print(f"`all_models_results` updated after {model_name_rf}.")
    except Exception as e:
        print(f"Error saving `all_models_results` after {model_name_rf}: {e}")

print("\n" + "="*50 + "\n")


--- Training Model: RandomForest_GridCV ---

===== Training RandomForest_GridCV for Target: EvaluationChoreographyStoryTelling =====
  Starting GridSearchCV for 'EvaluationChoreographyStoryTelling'...
  GridSearchCV completed in 38.47s. Best CV F1: 0.5653
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.7206, F1: 0.5489, Precision: 0.5438, Recall: 0.5540
  Holdout: CM (tn,fp,fn,tp): [859 271 260 323]

===== Training RandomForest_GridCV for Target: EvaluationChoreographyRhythm =====
  Starting GridSearchCV for 'EvaluationChoreographyRhythm'...
  GridSearchCV completed in 38.95s. Best CV F1: 0.6150
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.7243, F1: 0.5966, Precision: 0.6515, Recall: 0.5502
  Holdout: CM (tn,fp,fn,tp): [772 214 327 400]

===== Training RandomForest_GridCV for Target: EvaluationChoreographyMovementTechnique =====
  Starting GridSearchCV for 'EvaluationChoreographyMovementTechnique'...
  GridSearchCV completed in 39.91s. Best CV F1: 0.5514
  Metrics on Hold-Out

In [ ]:
# CELL XG: XGBOOST (GridSearchCV)
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import numpy as np 

model_name_xgb = "XGBoost_GridCV" 
if model_name_xgb not in all_models_results:
    all_models_results[model_name_xgb] = {}

print(f"\n--- Training Model: {model_name_xgb} ---")

.
xgb_param_grid = {
    'clf__n_estimators': [100, 200, 300],
    'clf__learning_rate': [0.01, 0.05, 0.1],
    'clf__max_depth': [3, 5, 7],
    'clf__subsample': [0.7, 0.9, 1.0],
    'clf__colsample_bytree': [0.7, 0.9, 1.0]
}

if 'target_columns' not in globals() or not target_columns:
    print(f"ERROR: 'target_columns' not defined/empty. Skipping {model_name_xgb} training.")
else:
    for target_name in target_columns:
        print(f"\n===== Training {model_name_xgb} for Target: {target_name} =====")

        if target_name not in data_splits_storage or data_splits_storage[target_name]['status'] != 'Split successful':
            status = data_splits_storage.get(target_name, {}).get('status', 'Split data not found or failed')
            print(f"  Skipping '{target_name}'. Reason: {status}")
            all_models_results[model_name_xgb][target_name] = {'status': status}
            continue

        X_train_80pct = data_splits_storage[target_name]['X_train']
        y_train_80pct = data_splits_storage[target_name]['y_train']
        X_holdout_20pct = data_splits_storage[target_name]['X_holdout']
        y_holdout_20pct = data_splits_storage[target_name]['y_holdout']

        min_class_count_train = y_train_80pct.value_counts().min()
        if min_class_count_train < N_CV_SPLITS:
            print(f"  WARNING: Smallest class ({min_class_count_train}) < N_CV_SPLITS ({N_CV_SPLITS}) for '{target_name}'. Skipping.")
            all_models_results[model_name_xgb][target_name] = {
                'status': f'Skipped - too few samples for {N_CV_SPLITS}-Fold CV ({min_class_count_train})'
            }
            continue

        # Calculate scale_pos_weight
        counts = y_train_80pct.value_counts()
        scale_pos_weight_val = counts.get(0, 1.0) / counts.get(1, 1.0) if counts.get(1, 0) > 0 else 1.0

        xgb_pipeline = Pipeline([
            ('imputer', KNNImputer(n_neighbors=5)),
            ('clf', XGBClassifier(objective='binary:logistic', eval_metric='logloss',
                                  use_label_encoder=False, random_state=RANDOM_STATE,
                                  scale_pos_weight=scale_pos_weight_val)) 
        ])

        cv_splitter = StratifiedKFold(n_splits=N_CV_SPLITS, shuffle=True, random_state=RANDOM_STATE)
        search_cv_xgb = GridSearchCV(
            xgb_pipeline, xgb_param_grid, cv=cv_splitter,
            scoring=SCORING_FOR_SKLEARN_CV_METHODS,
            refit=REFIT_METRIC_FOR_SKLEARN_CV,
            verbose=0, n_jobs=-1
        )

        print(f"  Starting GridSearchCV for '{target_name}'...")
        start_time_cv_xgb = time.time()
        try:
            search_cv_xgb.fit(X_train_80pct, y_train_80pct)
        except Exception as e:
            print(f"  ERROR during GridSearchCV for '{target_name}': {e}")
            all_models_results[model_name_xgb][target_name] = {'status': f'Skipped - SearchCV error: {e}'}
            continue
        cv_tuning_time_xgb = time.time() - start_time_cv_xgb
        print(f"  GridSearchCV completed in {cv_tuning_time_xgb:.2f}s. Best CV {REFIT_METRIC_FOR_SKLEARN_CV.upper()}: {search_cv_xgb.best_score_:.4f}")

        tuned_model_xgb = search_cv_xgb.best_estimator_
        tuned_model_filename = f"{model_name_xgb}_{target_name.replace(' ', '_')}.joblib"
        tuned_model_path = os.path.join(SAVED_TUNED_MODELS_DIR, tuned_model_filename)
        joblib.dump(tuned_model_xgb, tuned_model_path)

        y_train_pred_xgb = tuned_model_xgb.predict(X_train_80pct)
        y_train_proba_xgb = tuned_model_xgb.predict_proba(X_train_80pct)[:, 1]
        train_set_metrics_xgb = {
            'roc_auc': roc_auc_score(y_train_80pct, y_train_proba_xgb),
            'f1': f1_score(y_train_80pct, y_train_pred_xgb, zero_division=0),
            'precision': precision_score(y_train_80pct, y_train_pred_xgb, zero_division=0),
            'recall': recall_score(y_train_80pct, y_train_pred_xgb, zero_division=0),
            'confusion_matrix': confusion_matrix(y_train_80pct, y_train_pred_xgb)
        }

        start_time_inference_xgb = time.time()
        y_holdout_pred_xgb = tuned_model_xgb.predict(X_holdout_20pct)
        y_holdout_proba_xgb = tuned_model_xgb.predict_proba(X_holdout_20pct)[:, 1]
        inference_time_xgb = time.time() - start_time_inference_xgb
        holdout_set_metrics_xgb = {
            'roc_auc': roc_auc_score(y_holdout_20pct, y_holdout_proba_xgb),
            'f1': f1_score(y_holdout_20pct, y_holdout_pred_xgb, zero_division=0),
            'precision': precision_score(y_holdout_20pct, y_holdout_pred_xgb, zero_division=0),
            'recall': recall_score(y_holdout_20pct, y_holdout_pred_xgb, zero_division=0),
            'confusion_matrix': confusion_matrix(y_holdout_20pct, y_holdout_pred_xgb)
        }
        print("  Metrics on Hold-Out Set (20%):"); print_metrics(holdout_set_metrics_xgb, prefix="  Holdout: ")


        results_df_xgb = pd.DataFrame(search_cv_xgb.cv_results_)
        best_index_xgb = search_cv_xgb.best_index_
        fold_metrics_summary_xgb = {}
        for metric_key_cv_results in SCORING_FOR_SKLEARN_CV_METHODS.keys():
            fold_metrics_summary_xgb[metric_key_cv_results] = {
                'mean': results_df_xgb.iloc[best_index_xgb][f'mean_test_{metric_key_cv_results}'],
                'std': results_df_xgb.iloc[best_index_xgb][f'std_test_{metric_key_cv_results}']
            }


        all_models_results[model_name_xgb][target_name] = {
            'status': 'Completed', 'best_hyperparameters': search_cv_xgb.best_params_,
            'cv_mean_metrics': fold_metrics_summary_xgb,
            f'best_cv_{REFIT_METRIC_FOR_SKLEARN_CV}_score': search_cv_xgb.best_score_,
            'train_set_metrics': train_set_metrics_xgb, 'holdout_set_metrics': holdout_set_metrics_xgb,
            'cv_tuning_time_seconds': cv_tuning_time_xgb,
            'holdout_inference_time_seconds': inference_time_xgb,
            'saved_tuned_model_path': tuned_model_path,
            
        }

    print(f"\n--- Completed All Targets for {model_name_xgb} ---")
    try:
        joblib.dump(all_models_results, RESULTS_FILE_PATH)
        print(f"`all_models_results` updated after {model_name_xgb}.")
    except Exception as e:
        print(f"Error saving `all_models_results` after {model_name_xgb}: {e}")

print("\n" + "="*50 + "\n")


--- Training Model: XGBoost_GridCV ---

===== Training XGBoost_GridCV for Target: EvaluationChoreographyStoryTelling =====
  Starting GridSearchCV for 'EvaluationChoreographyStoryTelling'...


/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [21:52:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [21:52:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [21:52:36] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreograph

  GridSearchCV completed in 7.21s. Best CV F1: 0.5851
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.7246, F1: 0.5778, Precision: 0.5222, Recall: 0.6467
  Holdout: CM (tn,fp,fn,tp): [785 345 206 377]

===== Training XGBoost_GridCV for Target: EvaluationChoreographyRhythm =====
  Starting GridSearchCV for 'EvaluationChoreographyRhythm'...


/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [21:52:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [21:52:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [21:52:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreograph

  GridSearchCV completed in 8.83s. Best CV F1: 0.6245
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.7345, F1: 0.6315, Precision: 0.6553, Recall: 0.6094
  Holdout: CM (tn,fp,fn,tp): [753 233 284 443]

===== Training XGBoost_GridCV for Target: EvaluationChoreographyMovementTechnique =====
  Starting GridSearchCV for 'EvaluationChoreographyMovementTechnique'...


...

  GridSearchCV completed in 8.70s. Best CV F1: 0.5199
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.7002, F1: 0.5121, Precision: 0.4077, Recall: 0.6883
  Holdout: CM (tn,fp,fn,tp): [789 462 144 318]

===== Training XGBoost_GridCV for Target: EvaluationChoreographyHumanCharacterization =====
  Starting GridSearchCV for 'EvaluationChoreographyHumanCharacterization'...


/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [21:53:17] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [21:53:17] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [21:53:17] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreograph

  GridSearchCV completed in 7.35s. Best CV F1: 0.5473
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.5692, F1: 0.5437, Precision: 0.4376, Recall: 0.7178
  Holdout: CM (tn,fp,fn,tp): [371 644 197 501]

===== Training XGBoost_GridCV for Target: EvaluationChoreographyHumanReproducibility =====
  Starting GridSearchCV for 'EvaluationChoreographyHumanReproducibility'...


/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [21:53:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [21:53:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [21:53:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreograph

  GridSearchCV completed in 8.44s. Best CV F1: 0.8056
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.5539, F1: 0.8052, Precision: 0.7829, Recall: 0.8288
  Holdout: CM (tn,fp,fn,tp): [  97  302  225 1089]

--- Completed All Targets for XGBoost_GridCV ---
`all_models_results` updated after XGBoost_GridCV.




In [ ]:
# CELL CB: CATBOOST (GridSearchCV)
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import GridSearchCV, StratifiedKFold

model_name_cat = "CatBoost_GridCV" 
if model_name_cat not in all_models_results:
    all_models_results[model_name_cat] = {}

print(f"\n--- Training Model: {model_name_cat} ---")

cat_param_grid = {
    'clf__iterations': [100, 200, 300],    
    'clf__learning_rate': [0.01, 0.05, 0.1],
    'clf__depth': [3, 5, 7],
    'clf__l2_leaf_reg': [1, 3, 5, 7, 9],    
    'clf__auto_class_weights': ['Balanced', 'None'] 
}


if 'target_columns' not in globals() or not target_columns:
    print(f"ERROR: 'target_columns' not defined/empty. Skipping {model_name_cat} training.")
else:
    for target_name in target_columns:
        print(f"\n===== Training {model_name_cat} for Target: {target_name} =====")

        if target_name not in data_splits_storage or data_splits_storage[target_name]['status'] != 'Split successful':
            status = data_splits_storage.get(target_name, {}).get('status', 'Split data not found or failed')
            print(f"  Skipping '{target_name}'. Reason: {status}")
            all_models_results[model_name_cat][target_name] = {'status': status}
            continue

        X_train_80pct = data_splits_storage[target_name]['X_train']
        y_train_80pct = data_splits_storage[target_name]['y_train']
        X_holdout_20pct = data_splits_storage[target_name]['X_holdout']
        y_holdout_20pct = data_splits_storage[target_name]['y_holdout']

        min_class_count_train = y_train_80pct.value_counts().min()
        if min_class_count_train < N_CV_SPLITS:
            print(f"  WARNING: Smallest class ({min_class_count_train}) < N_CV_SPLITS ({N_CV_SPLITS}) for '{target_name}'. Skipping.")
            all_models_results[model_name_cat][target_name] = {
                'status': f'Skipped - too few samples for {N_CV_SPLITS}-Fold CV ({min_class_count_train})'
            }
            continue

        cat_pipeline = Pipeline([
            ('imputer', KNNImputer(n_neighbors=5)), 
            ('clf', CatBoostClassifier(random_state=RANDOM_STATE, verbose=0,
                                       
                                      ))
        ])

        cv_splitter = StratifiedKFold(n_splits=N_CV_SPLITS, shuffle=True, random_state=RANDOM_STATE)
        
     
        search_cv_cat = GridSearchCV(
            cat_pipeline, cat_param_grid, cv=cv_splitter,
            scoring=SCORING_FOR_SKLEARN_CV_METHODS,
            refit=REFIT_METRIC_FOR_SKLEARN_CV,
            verbose=0,
            n_jobs=-1
           
        )

        print(f"  Starting GridSearchCV for '{target_name}'...")
        start_time_cv_cat = time.time()
        try:
            search_cv_cat.fit(X_train_80pct, y_train_80pct)
        except Exception as e:
            print(f"  ERROR during GridSearchCV for '{target_name}': {e}")
            all_models_results[model_name_cat][target_name] = {'status': f'Skipped - SearchCV error: {e}'}
            continue
        cv_tuning_time_cat = time.time() - start_time_cv_cat
        print(f"  GridSearchCV completed in {cv_tuning_time_cat:.2f}s. Best CV {REFIT_METRIC_FOR_SKLEARN_CV.upper()}: {search_cv_cat.best_score_:.4f}")

        tuned_model_cat = search_cv_cat.best_estimator_
        tuned_model_filename = f"{model_name_cat}_{target_name.replace(' ', '_')}.joblib"
        tuned_model_path = os.path.join(SAVED_TUNED_MODELS_DIR, tuned_model_filename)
        joblib.dump(tuned_model_cat, tuned_model_path)

        y_train_pred_cat = tuned_model_cat.predict(X_train_80pct)
        y_train_proba_cat = tuned_model_cat.predict_proba(X_train_80pct)[:, 1]
        train_set_metrics_cat = {
            'roc_auc': roc_auc_score(y_train_80pct, y_train_proba_cat),
            'f1': f1_score(y_train_80pct, y_train_pred_cat, zero_division=0),
            'precision': precision_score(y_train_80pct, y_train_pred_cat, zero_division=0),
            'recall': recall_score(y_train_80pct, y_train_pred_cat, zero_division=0),
            'confusion_matrix': confusion_matrix(y_train_80pct, y_train_pred_cat)
        }

        start_time_inference_cat = time.time()
        y_holdout_pred_cat = tuned_model_cat.predict(X_holdout_20pct)
        y_holdout_proba_cat = tuned_model_cat.predict_proba(X_holdout_20pct)[:, 1]
        inference_time_cat = time.time() - start_time_inference_cat
        holdout_set_metrics_cat = {
            'roc_auc': roc_auc_score(y_holdout_20pct, y_holdout_proba_cat),
            'f1': f1_score(y_holdout_20pct, y_holdout_pred_cat, zero_division=0),
            'precision': precision_score(y_holdout_20pct, y_holdout_pred_cat, zero_division=0),
            'recall': recall_score(y_holdout_20pct, y_holdout_pred_cat, zero_division=0),
            'confusion_matrix': confusion_matrix(y_holdout_20pct, y_holdout_pred_cat)
        }
        print("  Metrics on Hold-Out Set (20%):"); print_metrics(holdout_set_metrics_cat, prefix="  Holdout: ")
        
        results_df_cat = pd.DataFrame(search_cv_cat.cv_results_)
        best_index_cat = search_cv_cat.best_index_
        fold_metrics_summary_cat = {}
        for metric_key_cv_results in SCORING_FOR_SKLEARN_CV_METHODS.keys():
            fold_metrics_summary_cat[metric_key_cv_results] = {
                'mean': results_df_cat.iloc[best_index_cat][f'mean_test_{metric_key_cv_results}'],
                'std': results_df_cat.iloc[best_index_cat][f'std_test_{metric_key_cv_results}']
            }

        all_models_results[model_name_cat][target_name] = {
            'status': 'Completed', 'best_hyperparameters': search_cv_cat.best_params_,
            'cv_mean_metrics': fold_metrics_summary_cat,
            f'best_cv_{REFIT_METRIC_FOR_SKLEARN_CV}_score': search_cv_cat.best_score_,
            'train_set_metrics': train_set_metrics_cat, 'holdout_set_metrics': holdout_set_metrics_cat,
            'cv_tuning_time_seconds': cv_tuning_time_cat,
            'holdout_inference_time_seconds': inference_time_cat,
            'saved_tuned_model_path': tuned_model_path,
        }

    print(f"\n--- Completed All Targets for {model_name_cat} ---")
    try:
        joblib.dump(all_models_results, RESULTS_FILE_PATH)
        print(f"`all_models_results` updated after {model_name_cat}.")
    except Exception as e:
        print(f"Error saving `all_models_results` after {model_name_cat}: {e}")

print("\n" + "="*50 + "\n")


--- Training Model: CatBoost_GridCV ---

===== Training CatBoost_GridCV for Target: EvaluationChoreographyStoryTelling =====
  Starting GridSearchCV for 'EvaluationChoreographyStoryTelling'...
  GridSearchCV completed in 35.41s. Best CV F1: 0.5863
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.7266, F1: 0.5747, Precision: 0.5217, Recall: 0.6398
  Holdout: CM (tn,fp,fn,tp): [788 342 210 373]

===== Training CatBoost_GridCV for Target: EvaluationChoreographyRhythm =====
  Starting GridSearchCV for 'EvaluationChoreographyRhythm'...
  GridSearchCV completed in 37.18s. Best CV F1: 0.6237
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.7280, F1: 0.6281, Precision: 0.6528, Recall: 0.6052
  Holdout: CM (tn,fp,fn,tp): [752 234 287 440]

===== Training CatBoost_GridCV for Target: EvaluationChoreographyMovementTechnique =====
  Starting GridSearchCV for 'EvaluationChoreographyMovementTechnique'...
  GridSearchCV completed in 34.25s. Best CV F1: 0.5631
  Metrics on Hold-Out Set (20%):
  Ho

/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_div

  GridSearchCV completed in 32.70s. Best CV F1: 0.5137
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.6985, F1: 0.5134, Precision: 0.4109, Recall: 0.6840
  Holdout: CM (tn,fp,fn,tp): [798 453 146 316]

===== Training CatBoost_GridCV for Target: EvaluationChoreographyHumanCharacterization =====
  Starting GridSearchCV for 'EvaluationChoreographyHumanCharacterization'...


/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ehsan/Desktop/industry/Evaluations_on_Robotic_Choreographies/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_div

  GridSearchCV completed in 32.12s. Best CV F1: 0.5587
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.5907, F1: 0.5332, Precision: 0.4574, Recall: 0.6390
  Holdout: CM (tn,fp,fn,tp): [486 529 252 446]

===== Training CatBoost_GridCV for Target: EvaluationChoreographyHumanReproducibility =====
  Starting GridSearchCV for 'EvaluationChoreographyHumanReproducibility'...
  GridSearchCV completed in 33.57s. Best CV F1: 0.8688
  Metrics on Hold-Out Set (20%):
  Holdout: AUC: 0.5779, F1: 0.8674, Precision: 0.7671, Recall: 0.9977
  Holdout: CM (tn,fp,fn,tp): [   1  398    3 1311]

--- Completed All Targets for CatBoost_GridCV ---
`all_models_results` updated after CatBoost_GridCV.




In [ ]:
# CELL FINAL: Results Compilation and Plotting
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import os
import joblib

print("\n--- Final Model Comparison: Compilation and Plotting Stage ---")

if 'RESULTS_FILE_PATH' not in globals(): 
    BASE_DIR = "." 
    RESULTS_FILE_PATH = os.path.join(BASE_DIR, "all_models_results_persistent.joblib")
    # Define other necessary globals if missing
    SAVED_OVERALL_BEST_MODELS_DIR = os.path.join(BASE_DIR, "saved_overall_best_models") # Used in next cell
    PRIMARY_METRIC_NAME = 'f1' 
    SCORING_FOR_SKLEARN_CV_METHODS = { # Used to iterate cv_mean_metrics keys
         'roc_auc': 'roc_auc', 'f1': 'f1', 'precision': 'precision', 'recall': 'recall'
    }
    print("Warning: Key global variables (e.g., RESULTS_FILE_PATH) not found, using defaults. Run CELL 0 for proper setup.")


if 'all_models_results' not in globals() or not all_models_results:
    print("Attempting to load `all_models_results` from disk...")
    try:
        all_models_results = joblib.load(RESULTS_FILE_PATH)
        print("Successfully loaded `all_models_results`.")
    except FileNotFoundError:
        print(f"ERROR: `all_models_results` not found at {RESULTS_FILE_PATH}. Please run previous model training cells.")
        all_models_results = {} 
    except Exception as e:
        print(f"ERROR loading `all_models_results`: {e}. Please run previous model training cells.")
        all_models_results = {}

comparison_data_list = []
if not all_models_results:
    print("`all_models_results` is not populated. Cannot compile comparison data.")
else:
    model_types_processed = sorted(list(all_models_results.keys())) 
    for model_type_key in model_types_processed:
        if model_type_key in all_models_results: 
            
            sorted_target_names = sorted(list(all_models_results[model_type_key].keys()))
            for target_name_original in sorted_target_names: 
                results_dict = all_models_results[model_type_key][target_name_original]
                if isinstance(results_dict, dict) and \
                   results_dict.get('status', '').startswith('Completed') and \
                   'holdout_set_metrics' in results_dict:
                    
                    holdout_metrics = results_dict['holdout_set_metrics']
                    train_metrics = results_dict.get('train_set_metrics', {})
                    cv_metrics = results_dict.get('cv_mean_metrics', {}) 
                    
                    display_target_name = target_name_original.replace("EvaluationChoreography", "")
                    if not display_target_name: display_target_name = target_name_original

                    cv_f1_mean = cv_metrics.get(PRIMARY_METRIC_NAME, {}).get('mean', np.nan)
                    cv_f1_std = cv_metrics.get(PRIMARY_METRIC_NAME, {}).get('std', np.nan)
                    cv_auc_mean = cv_metrics.get('roc_auc', {}).get('mean', np.nan)
                    cv_auc_std = cv_metrics.get('roc_auc', {}).get('std', np.nan)

                    comparison_data_list.append({
                        'Target': display_target_name, 
                        'Original Target Name': target_name_original, 
                        'Model Type': model_type_key,
                        'Holdout F1': holdout_metrics.get('f1', np.nan),
                        'Holdout AUC': holdout_metrics.get('roc_auc', np.nan),
                        'Holdout Precision': holdout_metrics.get('precision', np.nan),
                        'Holdout Recall': holdout_metrics.get('recall', np.nan),
                        'Train F1': train_metrics.get('f1', np.nan),
                        'Train AUC': train_metrics.get('roc_auc', np.nan),
                        'CV F1 Mean': cv_f1_mean,
                        'CV F1 Std': cv_f1_std,
                        'CV AUC Mean': cv_auc_mean,
                        'CV AUC Std': cv_auc_std,
                        'Saved Tuned Path': results_dict.get('saved_tuned_model_path', 'N/A'),
                    })

if not comparison_data_list:
    print("No completed model results found to compile or plot.")
    comparison_df = pd.DataFrame() 
else:
    comparison_df = pd.DataFrame(comparison_data_list)

    comparison_df = comparison_df.sort_values(by=['Target', 'Model Type'], ascending=[True, True])

    print("\n--- Overall Performance Summary (Holdout Set) ---")
 
    display_columns = ['Target', 'Model Type', 'Holdout F1', 'Holdout AUC', 'CV F1 Mean', 'CV F1 Std', 'CV AUC Mean', 'CV AUC Std']
    
    display_columns = [col for col in display_columns if col in comparison_df.columns]
    print(comparison_df[display_columns].to_string(float_format="%.4f"))

    # --- Plotting ---
    unique_targets_for_plot = sorted(comparison_df['Target'].unique())

    model_types_for_plot = sorted(comparison_df['Model Type'].unique()) 

    for metric_to_plot_label, metric_column_name in [('F1-Score', 'Holdout F1'), ('ROC AUC', 'Holdout AUC')]:
        fig = go.Figure()
        for mt_key in model_types_for_plot: # Iterate in sorted order
            model_type_data = comparison_df[comparison_df['Model Type'] == mt_key]
            if model_type_data.empty: continue
            

            plot_data_for_model = model_type_data.set_index('Target').reindex(unique_targets_for_plot)
            metric_values = plot_data_for_model[metric_column_name].tolist()
            
            fig.add_trace(go.Bar(name=mt_key, x=unique_targets_for_plot, y=metric_values,
                                 text=[f'{v:.3f}' if not pd.isna(v) else "N/A" for v in metric_values],
                                 textposition='auto'))
        fig.update_layout(
            title_text=f'Comparison: Holdout {metric_to_plot_label} per Target',
            xaxis_title='Target Variable', yaxis_title=metric_to_plot_label,
            barmode='group', legend_title_text='Model Type', 
            yaxis_range=[0, max(1.05, comparison_df[metric_column_name].max() * 1.1 if comparison_df[metric_column_name].notna().any() else 1.05) ] # Dynamic y-axis
        )
        fig.show()

print(f"\nCELL FINAL_A COMPLETED. `comparison_df` is ready with {len(comparison_df)} entries.")
print("Proceed to the next cell for interactive model selection.\n" + "="*50 + "\n")


--- Final Model Comparison: Compilation and Plotting Stage ---

--- Overall Performance Summary (Holdout Set) ---
                   Target                 Model Type  Holdout F1  Holdout AUC  CV F1 Mean  CV F1 Std  CV AUC Mean  CV AUC Std
0   HumanCharacterization            CatBoost_GridCV      0.5332       0.5907      0.5587     0.0093       0.6159      0.0033
7   HumanCharacterization  LogisticRegression_GridCV      0.4773       0.5334      0.4844     0.0020       0.5343      0.0032
14  HumanCharacterization        RandomForest_GridCV      0.4873       0.5972      0.4909     0.0173       0.6040      0.0067
21  HumanCharacterization             XGBoost_GridCV      0.5437       0.5692      0.5473     0.0104       0.5877      0.0107
1    HumanReproducibility            CatBoost_GridCV      0.8674       0.5779      0.8688     0.0003       0.5718      0.0074
8    HumanReproducibility  LogisticRegression_GridCV      0.6368       0.5321      0.6324     0.0048       0.5434      0.0088
15 


CELL FINAL_A COMPLETED. `comparison_df` is ready with 28 entries.
Proceed to the next cell for interactive model selection.



In [ ]:
# CELL FINAL_B: Interactive Model Selection and Saving

import os   
import shutil
import pandas as pd 
import numpy as np

# This cell assumes `comparison_df`, `all_models_results`, `SAVED_OVERALL_BEST_MODELS_DIR` 
# and `PRIMARY_METRIC_NAME` are available in memory from preceding cells (CELL 0 and CELL FINAL_A).

if 'comparison_df' not in globals() or comparison_df.empty:
    print("ERROR: `comparison_df` is not available or is empty. Please run 'CELL FINAL_A' first.")
else:
    print("\n\n--- Interactive Model Selection ---")
    user_selected_models = {}
    
    unique_targets_to_select = sorted(comparison_df['Target'].unique()) 
    
    for target_display_name_iter in unique_targets_to_select:
        print(f"\n\n===== Selecting model for Target: {target_display_name_iter} =====")
        
        target_models_df = comparison_df[
            comparison_df['Target'] == target_display_name_iter
        ].sort_values(
            by=['Holdout F1', 'Holdout AUC', 'Model Type'], 
            ascending=[False, False, True] 
        ).reset_index(drop=True) 
        
        if target_models_df.empty:
            print(f"  No models found for target '{target_display_name_iter}'. Skipping.")
            continue

        print("  Available models (sorted by Holdout F1 desc, then AUC desc, then Model Type asc):")
        
        models_for_choice_list = [] 
        for idx, row_data in target_models_df.iterrows(): 
            print(f"  {idx + 1}: {row_data['Model Type']} "
                  f"(Holdout F1: {row_data['Holdout F1']:.4f} [CV Mean: {row_data['CV F1 Mean']:.4f}, CV Std: {row_data['CV F1 Std']:.4f}], "
                  f"Holdout AUC: {row_data['Holdout AUC']:.4f} [CV Mean: {row_data['CV AUC Mean']:.4f}, CV Std: {row_data['CV AUC Std']:.4f}])")
            models_for_choice_list.append({
                'path': row_data['Saved Tuned Path'], 
                'model_type': row_data['Model Type'] 
            })

        while True:
            try:
                prompt_message = (f"  Enter number (1-{len(models_for_choice_list)}) of the model to save for '{target_display_name_iter}' "
                                  f"(or 0 to skip). Numbers refer to the list above for this run: ")
                choice_input = input(prompt_message)
                choice_idx_user = int(choice_input) 
                if 0 <= choice_idx_user <= len(models_for_choice_list):
                    break
                else:
                    print(f"  Invalid choice. Please enter a number between 0 and {len(models_for_choice_list)}.")
            except ValueError:
                print("  Invalid input. Please enter a number.")
            except EOFError: 
                print("  EOFError: No input received. Skipping this target.")
                choice_idx_user = 0 
                break
        
        if choice_idx_user > 0: 
            selected_model_details = models_for_choice_list[choice_idx_user - 1] 
            user_selected_models[target_display_name_iter] = { 
                'path': selected_model_details['path'],
                'model_type': selected_model_details['model_type']
            }
            print(f"  Selected '{selected_model_details['model_type']}' for '{target_display_name_iter}'.")
        else:
            print(f"  Skipped selecting a model for '{target_display_name_iter}'.")

    # --- Saving User-Selected Overall Best Models ---
    print("\n\n--- Saving User-Selected Overall Best Models ---")
    
    if 'SAVED_OVERALL_BEST_MODELS_DIR' not in globals():
        BASE_DIR = "." 
        SAVED_OVERALL_BEST_MODELS_DIR = os.path.join(BASE_DIR, "saved_overall_best_models")
        print(f"Warning: SAVED_OVERALL_BEST_MODELS_DIR was not in globals, defined as: {SAVED_OVERALL_BEST_MODELS_DIR}")
    
    os.makedirs(SAVED_OVERALL_BEST_MODELS_DIR, exist_ok=True)

    if not user_selected_models:
        print("  No models were selected by the user to save.")
    else:
        for target_display_name_saved, choice_info in user_selected_models.items():
            chosen_model_path = choice_info['path']
            model_type_in_filename = choice_info['model_type'] 
            
            if not chosen_model_path or chosen_model_path == 'N/A' or not os.path.exists(chosen_model_path):
                print(f"  ERROR: Invalid or missing model path for '{target_display_name_saved}' ('{chosen_model_path}'). Skipping.")
                continue

            base_name = os.path.basename(chosen_model_path)
            file_extension = os.path.splitext(base_name)[1]
            safe_target_name_part = "".join(c if c.isalnum() else "_" for c in target_display_name_saved)
            
            destination_filename = f"OVERALL_BEST_{model_type_in_filename}_{safe_target_name_part}{file_extension}"
            destination_path = os.path.join(SAVED_OVERALL_BEST_MODELS_DIR, destination_filename)

            try:
                shutil.copy2(chosen_model_path, destination_path) # shutil is now imported
                print(f"  Successfully saved user-selected model for '{target_display_name_saved}' ({model_type_in_filename}) to: {destination_path}")
            except Exception as e:
                print(f"  Error saving best model for '{target_display_name_saved}': {e}")

    print("\n\n--- Next Steps Example: SHAP Analysis (Conceptual) ---")
    print("For interpretability of your selected models:")
    print("1. Load an 'OVERALL_BEST' model from the 'saved_overall_best_models' directory.")
    print("2. Load its corresponding X_train/X_holdout data for the specific target (from the `classification_models_data` directory or `data_splits_storage`).")
    print("3. Use appropriate SHAP explainer (TreeExplainer or LinearExplainer).")

print("\nCELL FINAL_B: INTERACTIVE SELECTION AND SAVING COMPLETED.\n" + "="*50 + "\n")



--- Interactive Model Selection ---


===== Selecting model for Target: HumanCharacterization =====
  Available models (sorted by Holdout F1 desc, then AUC desc, then Model Type asc):
  1: XGBoost_GridCV (Holdout F1: 0.5437 [CV Mean: 0.5473, CV Std: 0.0104], Holdout AUC: 0.5692 [CV Mean: 0.5877, CV Std: 0.0107])
  2: CatBoost_GridCV (Holdout F1: 0.5332 [CV Mean: 0.5587, CV Std: 0.0093], Holdout AUC: 0.5907 [CV Mean: 0.6159, CV Std: 0.0033])
  3: RandomForest_GridCV (Holdout F1: 0.4873 [CV Mean: 0.4909, CV Std: 0.0173], Holdout AUC: 0.5972 [CV Mean: 0.6040, CV Std: 0.0067])
  4: LogisticRegression_GridCV (Holdout F1: 0.4773 [CV Mean: 0.4844, CV Std: 0.0020], Holdout AUC: 0.5334 [CV Mean: 0.5343, CV Std: 0.0032])
  Selected 'XGBoost_GridCV' for 'HumanCharacterization'.


===== Selecting model for Target: HumanReproducibility =====
  Available models (sorted by Holdout F1 desc, then AUC desc, then Model Type asc):
  1: CatBoost_GridCV (Holdout F1: 0.8674 [CV Mean: 0.8688, CV Std: 0.0003